# Lab 03 · Gemini Proxy Chat

*This lab notebook provides guided steps. All commands are intended for local execution.*

## Objectives
- A Gemini proxy endpoint is created in FastAPI.
- React chat UI components are connected to the proxy.
- API keys remain confined to the backend.

## What will be learned
- Backend proxy patterns for hosted LLMs are practiced.
- Basic chat state management in React is reviewed.
- Environment variable usage is reinforced.

## Prerequisites & install
The following commands are intended for local execution.

```bash
cd ai-web/backend
. .venv/bin/activate
pip install google-generativeai
```

## Step-by-step tasks
### Step 1: Tour the Gemini service module

Open `ai-web/backend/app/services/gemini.py` and guide students through the
implementation. The file is heavily commented so you can read it nearly line by
line:

```python
"""Service helpers for Gemini-powered features exposed by the FastAPI app."""

from __future__ import annotations

import os
from functools import lru_cache
from typing import List
```

- The module docstring reminds instructors that all Gemini logic should live in
  the service layer.
- `from __future__ import annotations` keeps type hints as strings at runtime so
  FastAPI imports stay lightweight.
- Standard-library imports (`os`, `lru_cache`, `typing.List`) support
  configuration, caching, and parsing.

```python
try:
    import google.generativeai as genai
except ImportError as exc:
    genai = None
    _IMPORT_ERROR = exc
else:
    _IMPORT_ERROR = None
```

- The `try/except` block lets unit tests run without the optional SDK. If the
  import fails, the service stores the exception and raises a friendly error when
  called.

```python
class GeminiServiceError(RuntimeError):
    """Raised when the Gemini helper cannot fulfill a request."""
```

- Custom exceptions make it easy for routers to translate domain failures into
  HTTP 503 responses.

```python
def _require_api_key() -> str:
    api_key = os.getenv("GEMINI_API_KEY")
    if not api_key:
        raise GeminiServiceError(
            "GEMINI_API_KEY is not configured. Add it to backend/.env before calling the service."
        )
    return api_key
```

- `_require_api_key` centralizes validation so every caller gets the same error
  message if the environment is missing.

```python
@lru_cache(maxsize=1)
def _configure_client(api_key: str) -> bool:
    if genai is None:
        raise GeminiServiceError(
            "google-generativeai is not installed. Run `pip install google-generativeai` to enable the feature."
        ) from _IMPORT_ERROR

    genai.configure(api_key=api_key)
    return True
```

- `@lru_cache` ensures the expensive `genai.configure` call only runs once per
  process. The guard re-raises the original import error with additional context.

```python
def _parse_outline_lines(raw_outline: str) -> List[str]:
    lines: List[str] = []
    for line in raw_outline.splitlines():
        cleaned = line.strip()
        if not cleaned:
            continue
        cleaned = cleaned.lstrip("-*•0123456789. \t")
        if cleaned:
            lines.append(cleaned)
    return lines
```

- `_parse_outline_lines` sanitizes model output by trimming blank lines and
  stripping bullets/numbers so the frontend receives plain text.

```python
def generate_lesson_outline(topic: str, model: str | None = None) -> dict[str, str | list[str]]:
    normalized_topic = topic.strip()
    if not normalized_topic:
        raise ValueError("Topic must not be empty.")

    api_key = _require_api_key()
    _configure_client(api_key)

    selected_model = model or os.getenv("GEMINI_MODEL", "gemini-2.5-flash")
    try:
        generative_model = genai.GenerativeModel(selected_model)
        prompt = (
            "You are helping an instructor design a web programming lesson. "
            "Return a concise outline with 3-5 bullet points that cover the key "
            "concepts for the topic: "
            f"{normalized_topic}."
        )
        response = generative_model.generate_content(prompt)
        outline_text = getattr(response, "text", "").strip()
    except Exception as exc:
        raw_message = str(exc).strip()
        if raw_message:
            detail = f": {raw_message}"
        else:
            fallback = exc.__class__.__name__
            detail = f": {fallback}"
        raise GeminiServiceError(
            f"Failed to generate lesson outline{detail}."
        ) from exc

    outline = _parse_outline_lines(outline_text) if outline_text else []
    return {"topic": normalized_topic, "outline": outline}
```

- The public function handles validation, client configuration, prompt creation,
  and error translation. Use this walkthrough to emphasize how services encapsulate
  third-party SDK calls while returning simple dictionaries to routers.


In [ ]:
from pathlib import Path
module = Path("ai-web/backend/app/llm.py")
module.parent.mkdir(parents=True, exist_ok=True)
module.write_text('''import os
from typing import Dict, List, Any

from google import genai


def chat(messages: List[Dict[str, Any]]) -> str:
  api_key = os.environ.get('GEMINI_API_KEY', '')
  if not api_key:
    raise RuntimeError('A backend API key is required.')
  client = genai.Client(api_key=api_key)
  response = client.models.generate_content(
      model="gemini-1.5-flash",
      contents=messages,
  )
  return response.text
''')
print("Gemini helper was written.")

### Step 2: Review the Gemini router

Next, inspect `ai-web/backend/app/routers/gemini.py` to see how the service is
exposed to the frontend.

```python
router = APIRouter(prefix="/ai", tags=["ai"])
```

- Namespacing routes under `/ai` keeps AI endpoints grouped together for
  documentation.

```python
class LessonOutlineIn(BaseModel):
    topic: constr(strip_whitespace=True, min_length=1)
```

- Input validation mirrors the service’s `strip()` call so bad requests surface
  as HTTP 422 responses before hitting Gemini.

```python
@router.post("/lesson-outline", response_model=LessonOutlineOut)
def lesson_outline(payload: LessonOutlineIn) -> LessonOutlineOut:
    try:
        result = generate_lesson_outline(payload.topic)
    except ValueError as exc:
        raise HTTPException(status_code=422, detail=str(exc)) from exc
    except GeminiServiceError as exc:
        logger.exception("Gemini lesson outline request failed")
        raise HTTPException(status_code=503, detail=str(exc)) from exc

    return LessonOutlineOut(**result)
```

- The route delegates to the service and translates domain errors into HTTP
  exceptions. Mention the `logger.exception` call so instructors remember to show
  console logs when debugging with the class.


In [ ]:
from pathlib import Path
main_path = Path("ai-web/backend/app/main.py")
text = main_path.read_text()
addition = '''
from typing import List
from pydantic import BaseModel
from .llm import chat as gemini_chat


class ChatTurn(BaseModel):
    role: str
    content: str


class ChatRequest(BaseModel):
    messages: List[ChatTurn]


@app.post("/api/chat")
def chat_endpoint(request: ChatRequest):
    transcript = [
        {"role": turn.role, "parts": [turn.content]} for turn in request.messages
    ]
    text = gemini_chat(transcript)
    return {"text": text}
'''
if "chat_endpoint" not in text:
    main_path.write_text(text.rstrip() + "
" + addition)
    print("FastAPI route was appended.")
else:
    print("FastAPI route already present.")

### Step 3: Demonstrate the Gemini lesson-outline feature

Show how the frontend consumes the new endpoint:

1. **Hook (`features/gemini/hooks/useLessonOutlineForm.js`)** – Posts the topic
   to `/ai/lesson-outline`, manages loading/error state, and renders any structured
   outline returned by the backend. Compare its structure to `useEchoForm` so the
   shared pattern is obvious.
2. **Component (`features/gemini/components/LessonOutlineForm.jsx`)** – Presents
   the instructional copy, form fields, and formatted outline. Highlight how the
   component reads `form.outline` to create a bulleted list.
3. **App shell (`src/App.jsx`)** – Already renders the Gemini section under the
   heading “Gemini lesson outline builder.” Reinforce that all AI features should
   follow this composition model.

Encourage instructors to run the flow live, pointing out the loading indicator
and error alert when the backend raises `GeminiServiceError` (for example, by
commenting out `GEMINI_API_KEY`).


In [ ]:
from pathlib import Path
app_js = Path("ai-web/frontend/src/App.jsx")
app_js.write_text('''import React, { useState } from 'react';
import { post } from './lib/api';
import { withRetry } from './lib/retry';

function App() {
  const [messages, setMessages] = useState([{ role: 'user', content: 'Hello Gemini' }]);
  const [input, setInput] = useState('');
  const [loading, setLoading] = useState(false);
  const [error, setError] = useState('');

  async function handleSend(event) {
    event.preventDefault();
    const updated = [...messages, { role: 'user', content: input }];
    setMessages(updated);
    setLoading(true);
    setError('');
    try {
      const result = await withRetry(
        () => post('/api/chat', { messages: updated }),
        1,
        800
      );
      setMessages([...updated, { role: 'model', content: result.text }]);
      setInput('');
    } catch (err) {
      setError('The proxy was unreachable. Please try again.');
    } finally {
      setLoading(false);
    }
  }

  return (
    <main style={{ padding: 24 }}>
      <h1>Lab 3 — Gemini proxy chat</h1>
      <section>
        {messages.map((msg, index) => (
          <p key={index}>
            <strong>{msg.role}:</strong> {msg.content}
          </p>
        ))}
      </section>
      <form onSubmit={handleSend}>
        <input
          value={input}
          onChange={(event) => setInput(event.target.value)}
          placeholder="Type a follow-up"
        />
        <button type="submit" disabled={loading || !input}>Send</button>
      </form>
      {loading && <p>Awaiting proxy response…</p>}
      {error && <p style={{ color: 'red' }}>{error}</p>}
    </main>
  );
}

export default App;
''')
print("Chat UI was seeded.")

## Validation / acceptance checks
```bash
# locally
export GEMINI_API_KEY="<your real key>"
uvicorn app.main:app --reload
curl -X POST http://localhost:8000/ai/lesson-outline   -H 'Content-Type: application/json'   -d '{"topic":"State management in React"}'
```
- Response JSON includes the normalized topic and an `outline` array.
- Frontend form displays bullet points that match the backend payload.
- If `GEMINI_API_KEY` or the SDK import is missing, the API responds with HTTP 503
  and surfaces the descriptive error message from `GeminiServiceError`.


## Homework / extensions
- Streaming responses are researched for future enhancements.
- Chat history persistence is sketched for the next lab.